In [1]:
import os
import pathlib

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf 
import tensorflow_io as tfio

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

import librosa
import librosa.display
import IPython


In [8]:
DATASET_PATH = 'data_aug_img'
data_dir = pathlib.Path(DATASET_PATH)

ODCZYTANIE I PODZIAŁ DANYCH WEJŚCIOWYCH

In [13]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

filenames = tf.io.gfile.glob(str(data_dir)+'/*/*.png')
filenames = tf.random.shuffle(filenames)

files_total_number = len(filenames)
print('Liczba wszystkich próbek : ', files_total_number)

commands = np.array(tf.io.gfile.listdir(str(data_dir)))

for command in commands:
    print(command, len(tf.io.gfile.glob( str(os.path.join(data_dir,command)) + '/*.png')))

Liczba wszystkich próbek :  240
00-odrzuc_bron 48
01-obroc_sie 48
02-na_kolana 48
03-gleba 48
04-rece_na_glowe 48


In [12]:
train_num = 0.70
val_num = 0.15
test_num = 0.15

train_files = filenames[:int(files_total_number*train_num)]
val_files = filenames[int(files_total_number*train_num) : int(files_total_number*train_num) + int(files_total_number*val_num)]
test_files = filenames[int(files_total_number*train_num) + int(files_total_number*val_num):]


print('Zestaw uczący : ', len(train_files))
print('Zestaw walidacyjny : ', len(val_files))
print('Zestaw testowy : ', len(test_files))


Zestaw uczący :  168
Zestaw walidacyjny :  36
Zestaw testowy :  36


In [21]:
def get_label(file_path):
    parts = tf.strings.split( input=file_path, sep=os.path.sep )
    # Note: You'll use indexing here instead of tuple unpacking to enable this
    # to work in a TensorFlow graph.
    return parts[-2]

In [22]:
def load_spec_img(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=(100, 100)) # optional
    label = tf.cast(tf.equal(label, 'class2'), tf.int32)
    return img, label